# Evaluate model performance

In [1]:
# Load price data
import pandas as pd
prices = pd.read_csv('data/eurusd_minute.csv')['BidClose'].values

In [4]:
# Sample standardized data interval values
from pyexpat import model
from dataset import EncodedForexData
import numpy as np
from tqdm import tqdm

from encoders import SimpleRasterizeEncoder, GasfEncoder, RGGafEncoder

ENCODER = 'simple'

params = {
    'simple': {'encoder':SimpleRasterizeEncoder(), 'channels':1, 'relative':True},
    'gasf': {'encoder':GasfEncoder(), 'channels':1, 'relative':False},
    'rggaf': {'encoder':RGGafEncoder(), 'channels':2, 'relative':True},
}[ENCODER]

dataset = EncodedForexData(prices, 64, encoder=params['encoder'], relative=params['relative'])
encoder = params['encoder']


standardize = lambda x: (x-x.mean())/x.std()
decode = lambda x: standardize(encoder.decode(x))

data = np.concatenate([decode(mat) for mat in tqdm(dataset)], axis=0)

100%|██████████| 80690/80690 [00:06<00:00, 13350.32it/s]


In [5]:
from models import Generator
model = Generator(channels=params['channels'])

In [3]:
import matplotlib.pyplot as plt
def plot_cdf(data):
    y = 1. * np.arange(len(data)) / (len(data) - 1)
    plt.plot(np.sort(data), y)

In [13]:
# Compare generated data to original data
from transforms import reverse_rel_raster, reverse_gasf
from dataset import ForexData
import numpy as np
import matplotlib.pyplot as plt

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def evaluate(model, decoder, n):
    model.eval()
    with torch.no_grad():
        noise = torch.randn(n,100).to(device)
        encoded = model(noise)
    model_samples = decoder(encoded)

    model_data = model_samples.reshape(-1)
    plot_cdf(model_data)
    plot_cdf(model_data)
    plt.show()

In [ ]:
for weights in []:
    model.load_state_dict(torch.load(weights))
    evaluate(model, decode, 100)